In [1]:
import random
import numpy as np
from urllib.request import urlopen
import re

In [2]:
random.seed(42)

In [3]:
a = []
a.append([])
a[0].append(1)
a.append([])
a.append([])
a.append([])
a[1].append(4)
print(a)

[[1], [4], [], []]


In [12]:
adj_list = []
edge_len = []
receptors = []
transcription_factors = []

proteins = {}
curr_node = 0

In [13]:
fpointer = open("./datasets/4932.protein.links.v11.5.txt")
for line in fpointer:
    temp_list = line.split(" ")
    protein1 = temp_list[0].split(".")[1]
    protein2 = temp_list[1].split(".")[1]
    
    if(protein1 not in proteins.keys()):
        proteins[protein1] = curr_node
        curr_node += 1
        adj_list.append([])
        edge_len.append([])
    
    if(protein2 not in proteins.keys()):
        proteins[protein2] = curr_node
        curr_node += 1
        adj_list.append([])
        edge_len.append([])
    
    adj_list[proteins[protein1]].append(proteins[protein2])
    adj_list[proteins[protein2]].append(proteins[protein1])
    
    edge_len[proteins[protein1]].append(proteins[protein2])
    edge_len[proteins[protein2]].append(proteins[protein1])
    

In [ ]:
# print(proteins)
# print(proteins)
# print(len(adj_list))

In [ ]:
fout = open('./datasets/protein_list.txt','w')
for protein in proteins.keys():
    fout.write(f"{protein}\n")
fout.close()

In [15]:
num_v = len(proteins)
k = 8
v = np.arange(num_v)
v_colors = [random.randint(0,k-1) for i in v]

In [ ]:
x = 0
flag = 0
for protein in proteins.keys():
    if protein == "YOL002C":
        flag = 1
        continue
    if flag == 0:
        continue
    x += 1
    if x%5 == 0 :
        print(x)
    url = 'http://www.yeastract.com/view.php?existing=protein&proteinname=' + protein
    html_content = str(urlopen(url).read())
    matches = re.findall('Transcription&nbsp;Factor', html_content)
    if len(matches) > 0:
        transcription_factors.append(protein)
        # print(protein, 'T')
    matches = re.findall('receptor', html_content)
    if len(matches) > 0:
        receptors.append(protein)
        # print(protein, 'R')

# print(transcription_factors)

In [28]:
f = open("transcriptors.txt", "w")
for t in transcription_factors:
    f.write(t + "\n")
f.close()

f = open("receptors.txt", "w")
for r in receptors:
    f.write(r + "\n")
f.close()

In [9]:
def dfs(transcription_factors, adj_list, edge_probs, visited_colors, next_in_path, num_visited_colors, u, k, max_length, curr_length):
    visited_colors[v_colors[u]] = 1
    num_visited_colors += 1
    if(num_visited_colors == k and u in transcription_factors and curr_length > max_length):
        max_length = curr_length
        return True
    
    flag = False

    for v in adj_list[u]:
        if(visited_colors[v_colors[v]] == 1):
            continue
        flag = dfs(transcription_factors, adj_list, edge_probs, visited_colors, next_in_path, num_visited_colors, v, k, max_length, curr_length + edge_probs[u][v])
        if(flag):
            next_in_path[u] = v
    
    visited_colors[v_colors[u]] = 0
    num_visited_colors -= 1

    return flag 
